In [2]:
import networkx as nx
from networkx.algorithms import community

edgelist = []
with open("amazon-meta.txt", "r", encoding="utf8") as a_file:
    for line in a_file:
        stripped_line = line.strip()
        if stripped_line[0:5] == 'ASIN:':
            cur_item = stripped_line[5:].strip()
        if stripped_line[0:8] == 'similar:':
            cur_tuple = stripped_line[9:]
            cur_list = cur_tuple.split('  ')
            cur_list.pop(0)
            if len(cur_list) != 0:
                for i in cur_list:
                    edgelist.append(str(cur_item) + ' ' + i)
                    
    G = nx.parse_edgelist(edgelist)

In [3]:
import community as community_louvain

# partition is a dictionary that uses the node as a key, and the number of the community it belongs to as it's value
partition = community_louvain.best_partition(G)

In [4]:
i = 0
for key in partition:
    if i > 1000 and i < 1020: print( key, ' -> ', partition[key])
    i+=1

B0001XP2T0  ->  85
B0000073HU  ->  85
B0000073HS  ->  85
6301967917  ->  15
B0006TPDPM  ->  15
B0003JANSW  ->  15
B000063UR5  ->  15
B00008LDNZ  ->  15
B0006HBZBK  ->  15
0027348407  ->  28
014030956X  ->  28
0140303588  ->  28
0525304002  ->  28
0440486815  ->  28
0525392440  ->  28
0872886476  ->  21
158923121X  ->  21
0872885259  ->  21
0830641424  ->  21


In [5]:
# the communities dictionary indexes the communities and uses its index as the key and the array in that community as its value
communities = {}

for key in partition:
    if partition[key] in communities.keys():
        communities[partition[key]].append(key)
    else:
        communities[partition[key]] = [key]

i = 0
for community in communities:
    if i < 5 and len(communities[community]) < 20: 
        # returning a couple of communities that don't have a lot of communities for better visability
        print(community, ' -> ', communities[community])
        i+=1

36  ->  ['B0000060T5', '6303953344', '630395331X', '6303953328', '6303953301', '6304159927', '6304186851', '6304186827']
62  ->  ['0590108859', '0439043638', '0590138596', '0590216872', '0439992575', '043999232X', '0590138588', '043904362X', '0439981069', '0439999278']
66  ->  ['0553487388', '0553487094', '0553487108', '055348737X', '0553486802', '0553486799', '0553486829', '0553486810', '0553487078']
77  ->  ['B000003ONN', 'B00004RBUH', 'B000002ZAG', 'B0000BXBXG']
81  ->  ['0824799372', '0750675691', '0878147187']


In [6]:
# all_communities is an array of all the arrays of communities
all_communities = []
for community in communities:
    all_communities.append(communities[community])
    
# sorting communities 
all_communities.sort(key=len)
all_communities.reverse()

i = 0
while i < 11:
    print(len(all_communities[i])) # returns the length of the top 11 communities
    i+=1

# we returned the top 11 so we can see that the 10th and 11th largest don't have the same size
# we then made a new partition with just the top 10 communities
top_10 = {}
for community in communities:
    if (len(communities[community]) >= 9911):
        top_10[community] = communities[community]

28034
17316
16076
15087
12988
11634
11119
10487
10191
9968
9900


In [7]:
# turn top 10 communities into a partition
top_10_partition = {}
for com in top_10:
    for node in top_10[com]:
        top_10_partition[node] = com
    print (node, com)

0849960630 0
0151005605 6
1568811241 11
0801847575 14
B00006JBIX 15
0060537612 17
1878990950 20
B00006RGI2 26
0521421179 45
1890318426 57


In [8]:
print(community_louvain.modularity(partition, G))
# print(community_louvain.modularity(top_10_partition, G))

0.9244989523059571


In [9]:
# Gets PageRank, returns a dictionary of nodes with its pagerank as the value
pr = nx.pagerank(G)

In [13]:
i = 0
for p in pr:
    if i < 5:
        print(p, pr[p])
    i += 1        

0827229534 1.7455492274617236e-06
0804215715 7.626854440762131e-06
156101074X 5.986557107965901e-06
0687023955 4.209430170790477e-06
0687074231 2.5510851820318872e-06


In [22]:
sorted_values = sorted(pr.values(), reverse=True)
sorted_dict = {}

x = 0
for i in sorted_values:
    if x < 10:
        for j in pr.keys():
            if pr[j] == i:
                sorted_dict[j] = pr[j]
                break
        x += 1
print(sorted_dict)
# for value

{'B0002IVN9W': 9.253275475427091e-05, 'B00008LDNZ': 8.10392800384962e-05, 'B00023P4I8': 7.956559859339843e-05, '0670033375': 6.873713601221831e-05, '0890420254': 6.811337667598832e-05, '0679454438': 6.34032626136723e-05, '1582701148': 5.91598094527728e-05, '1557987912': 5.909664058255473e-05, '1594200246': 5.530857546474262e-05, '0393050572': 5.473392479323347e-05}


In [3]:
# import matplotlib.cm as cm
# import matplotlib.pyplot as plt
# # draw the graph
# pos = nx.spring_layout(G)
# # color the nodes according to their partition
# cmap = cm.get_cmap('viridis', max(partition.values()) + 1)
# nx.draw_networkx_nodes(G, pos, partition.keys(), node_size=40, cmap=cmap, node_color=list(partition.values()))
# nx.draw_networkx_edges(G, pos, alpha=0.25)
# plt.show()